# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [2]:
# Your code here; create the connection and cursor
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [7]:
cur.execute('''SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';''')
df = pd.DataFrame(cur.fetchall())
df.columns = ['customerNumber', 'contactLastName', 'contactFirstName']
df.head()

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [8]:
# Your code here; use a subquery. No join is necessary
cur.execute('''SELECT customerNumber, contactLastName, contactFirstName
                FROM customers
                WHERE customerNumber 
                IN (SELECT customerNumber FROM orders WHERE orderDate = '2003-01-31');''')
df = pd.DataFrame(cur.fetchall())
df.columns = ['customerNumber', 'contactLastName', 'contactFirstName']
df.head()

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [28]:
# Your code here
cur.execute('''SELECT productName, SUM(quantityOrdered) AS itemSold
                FROM(SELECT * FROM products JOIN orderdetails USING (productCode))
                GROUP BY (productName)
                ORDER BY itemSold DESC;''')
df = pd.DataFrame(cur.fetchall())
df.columns = ['productName', 'itemSold']
df.head()

,productName,itemSold
0,1992 Ferrari 360 Spider red,1808
1,1937 Lincoln Berline,1111
2,American Airlines: MD-11S,1085
3,1941 Chevrolet Special Deluxe Cabriolet,1076
4,1930 Buick Marquette Phaeton,1074


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [50]:
cur.execute('''SELECT productName, COUNT(customerNumber) AS itemSold FROM customers JOIN
                (SELECT * FROM orders JOIN 
                (SELECT * FROM products JOIN orderdetails USING (productCode)) 
                USING (orderNumber))
                USING (customerNumber)
                GROUP BY productName
                ORDER BY itemSold DESC;''')
df = pd.DataFrame(cur.fetchall())
df.columns = ['productName', 'itemSold']
df.head()

,productName,itemSold
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28


### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [51]:
# Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
cur.execute('''SELECT DISTINCT customerNumber
                FROM orders''')
df = pd.DataFrame(cur.fetchall())
df.columns = ['customerNumber']
df.head()

,customerNumber
0,363
1,128
2,181
3,121
4,141


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [71]:
cur.execute('''SELECT DISTINCT employeeNumber, lastName, firstName, city, officeCode, totalOrders 
                FROM offices JOIN(
                SELECT employeeNumber, lastName, firstName, officeCode, SUM(a) AS totalOrders from employees JOIN (
                SELECT customerNumber, COUNT(orderNumber) AS a, salesRepEmployeeNumber from customers 
                JOIN orders USING (customerNumber)
                GROUP BY customerNumber)
                ON employeeNumber=salesRepEmployeeNumber
                GROUP BY employeeNumber
                HAVING totalOrders < 20)
                USING (officeCode)
                ORDER BY totalOrders DESC;''')
df = pd.DataFrame(cur.fetchall())
df.columns = ['employeeNumber', 'lastName', 'firstName', 'city', 'officeCode', 'totalOrders']
df.head()

,employeeNumber,lastName,firstName,city,officeCode,totalOrders
0,1611,Fixter,Andy,Sydney,6,19
1,1612,Marsh,Peter,Sydney,6,19
2,1216,Patterson,Steve,Boston,2,18
3,1286,Tseng,Foon Yue,NYC,3,17
4,1621,Nishi,Mami,Tokyo,5,16


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [79]:
# Your code here
cur.execute('''SElECT employeeNumber, lastName, firstName, number
                FROM employees JOIN (
                SELECT salesRepEmployeeNumber, COUNT(salesRepEmployeeNumber) AS number FROM customers
                WHERE creditLimit > 15000
                GROUP BY salesRepEmployeeNumber)
                ON employeeNumber=salesRepEmployeeNumber
                ORDER BY number DESC;''')
df = pd.DataFrame(cur.fetchall())
df.columns = ['employeeNumber', 'lastName', 'firstName', 'number']
df.head()

,employeeNumber,lastName,firstName,number
0,1401,Castillo,Pamela,10
1,1504,Jones,Barry,9
2,1323,Vanauf,George,8
3,1501,Bott,Larry,8
4,1370,Hernandez,Gerard,7


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!